In [1]:
# !pip install pulp

In [2]:
import pulp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# to display all rows columns 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [3]:
df = pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')

In [4]:
df = df.join(pd.get_dummies(df[['Role', 'Team']]))
df['PlayingXI'] = 1

teams = list(df.Team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Sel'] for obj in df.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team1}"]] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team1}"]] for obj in df.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team2}"]] for obj in df.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team2}"]] for obj in df.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4

mo.solve()

# output as a column in df
df['pickup_status'] = 0

for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [5]:
sum(fdf.Credits)

99.5

In [9]:
fdf[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'pickup_status']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,pickup_status
0,S Iyer,9428,IND,BAT,71.23,8.5,0,1,1
1,C Asalanka,10934,SL,BAT,64.60,8.5,0,1,1
2,R Sharma,576,IND,BAT,87.30,10.5,0,1,1
3,P Nissanka,13682,SL,BAT,85.62,9.5,0,1,1
4,I Kishan,10276,IND,WK,73.23,8.5,0,1,1
5,K Mendis,9494,SL,WK,53.58,9.0,0,1,1
6,R Jadeja,587,IND,AR,74.73,9.0,0,1,1
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,1
8,D Chameera,8393,SL,BOWL,78.22,9.0,0,1,1
9,H Patel,8175,IND,BOWL,65.20,8.5,0,1,1


### Post Match Analysis

In [10]:
fdf[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'Points', 'DT', 'pickup_status']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,pickup_status
0,S Iyer,9428,IND,BAT,71.23,8.5,0,1,84,1,1
1,C Asalanka,10934,SL,BAT,64.60,8.5,0,1,70,1,1
2,R Sharma,576,IND,BAT,87.30,10.5,0,1,66,1,1
3,P Nissanka,13682,SL,BAT,85.62,9.5,0,1,2,0,1
4,I Kishan,10276,IND,WK,73.23,8.5,0,1,141,1,1
5,K Mendis,9494,SL,WK,53.58,9.0,0,1,0,0,1
6,R Jadeja,587,IND,AR,74.73,9.0,0,1,34,1,1
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,0,0,1
8,D Chameera,8393,SL,BOWL,78.22,9.0,0,1,30,1,1
9,H Patel,8175,IND,BOWL,65.20,8.5,0,1,8,0,1
